# https://keras.io/examples/audio/ctc_asr/

In [21]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# tf.compat.v1.disable_eager_execution()
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Layer
# from tensorflow.keras.layers import Input, Dense, Activation, LeakyReLU, Permute, Bidirectional
# from tensorflow.keras.layers import Reshape, Lambda, BatchNormalization, LSTM
# from tensorflow.python.keras.layers.merge import add, concatenate
# from tensorflow.python.keras.layers.recurrent import LSTM
# from tensorflow.keras.layers import CuDNNLSTM
from primus import CTC_PriMuS
import ctc_utils
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [22]:
tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[0], True)

In [23]:
def default_model_params(img_height, vocabulary_size):
    params = dict()
    params['img_height'] = img_height
    params['img_width'] = None
    params['batch_size'] = 16
    params['img_channels'] = 1
    params['conv_blocks'] = 2
    params['conv_filter_n'] = [32, 64]
    params['conv_filter_size'] = [ [3,3], [3,3] ]
    params['conv_pooling_size'] = [ [2,2], [2,2] ]
    params['rnn_units'] = 32
    params['rnn_layers'] = 1
    params['vocabulary_size'] = vocabulary_size
    return params

In [24]:
def CTCLoss(y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64") # 16
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        print("Y_pred shape:  ",y_pred.shape)
        print("Y_true shape:  ",y_true.shape)

        loss = K.ctc_batch_cost(y_true, y_pred, input_length, label_length)
        return loss

In [25]:
#model
def buildModel(params, input_width, rnn_layers = 1, rnn_units = 64):
    input_shape = (params['img_height'],params['img_width'], params['img_channels'])

    
    inputs = layers.Input(type_spec=tf.TensorSpec(shape=[None, params['img_height'], None, 1], dtype=tf.float32), name = "image" )
    # inputs = layers.Input(name='image', shape=input_shape, dtype='float32')

    labels = layers.Input(type_spec=tf.TensorSpec(shape=[None, None], dtype=tf.float32), name = "label" )
    # x = layers.Reshape((params['img_height'], input_width, 1), name="expand_dim")(inputs)
    x = layers.Conv2D(
        filters=32,
        kernel_size= [params['conv_filter_n'][0], params['conv_filter_n'][0]],
        strides=[2, 2],
        padding="same",
        use_bias=False,
        name="conv_1",
    )(inputs)
    x = layers.BatchNormalization(name="conv_1_bn")(x)
    x = layers.ReLU(name="conv_1_relu")(x)
    x = layers.MaxPool2D(pool_size=params['conv_pooling_size'][0], strides = params['conv_pooling_size'][0], name='max1')(x)

    x = layers.Conv2D(
        filters=64,
        kernel_size=[params['conv_filter_n'][1], params['conv_filter_n'][1]],
        strides=[1, 2],
        padding="same",
        use_bias=False,
        name="conv_2",
    )(x)
    x = layers.BatchNormalization(name="conv_2_bn")(x)
    x = layers.ReLU(name="conv_2_relu")(x)
    x = layers.MaxPool2D(pool_size=params['conv_pooling_size'][1], strides = params['conv_pooling_size'][1], name='max2')(x)
    print(x.shape)
    x = layers.Reshape((-1, x.shape[-3] * x.shape[-1]))(x)

    for i in range(1, rnn_layers + 1):
        recurrent = layers.GRU(
            units=rnn_units,
            activation="tanh",
            recurrent_activation="sigmoid",
            use_bias=True,
            return_sequences=True,
            reset_after=True,
            name=f"gru_{i}",
        )
        x = layers.Bidirectional(
            recurrent, name=f"bidirectional_{i}", merge_mode="concat"
        )(x)
        if i < rnn_layers:
            x = layers.Dropout(rate=0.5)(x)

    num_classes = params['vocabulary_size'] + 1
    y_pred = layers.Dense(num_classes, kernel_initializer='he_normal',name='dense2', activation='softmax')(x) #(None, 32, 63)        

    model = Model([inputs, labels], y_pred, name="OMR")
    opt = Adam(learning_rate=1e-4)
    model.compile(optimizer=opt, loss=CTCLoss)
    return model

In [26]:
img_height = 64
corpus_dirpath = "C:/Users/steli/DeepLearning/Project/tf-end-to-end/Data/primusCalvoRizoAppliedSciences2018/"
corpus_filepath = "C:/Users/steli/DeepLearning/Project/tf-end-to-end/Data/train.txt"
dictionary_path = "C:/Users/steli/DeepLearning/Project/tf-end-to-end/Data/vocabulary_semantic.txt"
primus = CTC_PriMuS(corpus_dirpath,corpus_filepath,dictionary_path, True, val_split = 0.2)
# corpus_dirpath = "/content/primusCalvoRizoAppliedSciences2018/"
# corpus_filepath = "/content/train.txt"
# dictionary_path = "/content/vocabulary_semantic.txt"


Training with 800 and validating with 200


In [27]:
# Get Data
image_paths = []
image_texts = []

class PriMuS_Data:
    def __init__(self, corpus_dirpath, corpus_filepath, dictionary_path):
        self.corpus_dirpath = corpus_dirpath

        # Corpus
        corpus_file = open(corpus_filepath,'r')
        self.corpus_list = corpus_file.read().splitlines()
        corpus_file.close()

        self.current_idx = 0

        # Dictionary
        self.word2int = {}
        self.int2word = {}
            
        dict_file = open(dictionary_path,'r')
        dict_list = dict_file.read().splitlines()
        for word in dict_list:
            if not word in self.word2int:
                word_idx = len(self.word2int)
                self.word2int[word] = word_idx
                self.int2word[word_idx] = word

        dict_file.close()

        self.vocabulary_size = len(self.word2int)
    
    def get_vocabulary_size(self):
        return self.vocabulary_size
        
    def get_data(self):
        return self.corpus_list

    def encode_seqs(self, seqs):
        encoded = []
        for seq in seqs:
            new_seq=[]
            for sym in seq:
                new_seq.append(self.word2int[sym])
            encoded.append(new_seq)
        return encoded

    def trim_ds(self):
        val_split = 0.9
        folder_path = self.corpus_dirpath
        list_of_files = self.corpus_list #10,000 filenames

        image_paths = [self.corpus_dirpath+f"{x}/{x}.png" for x in self.corpus_list] # list of image paths
        text_paths = [self.corpus_dirpath+f"{x}/{x}.semantic" for x in self.corpus_list]

        image_texts = [] # list of strings

        for path in text_paths:
            with open(path, "r") as file:
                image_texts.append(file.readline().split())

        image_texts = self.encode_seqs(image_texts)

        max_label_len = max([len(seq) for seq in image_texts])
        padded_image_texts = pad_sequences(image_texts, maxlen=max_label_len, padding='post', value= self.vocabulary_size + 1)

        # TODO: do line 17 from the link : https://github.com/rajesh-bhat/spark-ai-summit-2020-text-extraction/blob/master/CRNN_CTC_wandb.ipynb

        # TODO: Use self.word2int to encode each element of the lists in image_texts
        # Input image_texts = [  ['clef','C-note'] , [.......], ....]
        # Final image_texts = [  [5,3] , [.......], ....]
        # if 'clef' == 5, 'C-note' == 3 in word2int dictionary
        # Also add padding using the function below so length of each element is consistent

        # from tensorflow.keras.preprocessing.sequence import pad_sequences
        # padded_image_texts = list(map(encode_to_labels, image_texts))
        # padded_image_texts[0]


        train_image_paths = image_paths[ : int(len(image_paths) * val_split)]
        train_image_texts = padded_image_texts[ : int(len(padded_image_texts) * val_split)]

        val_image_paths = image_paths[int(len(image_paths) * val_split) : ]
        val_image_texts = padded_image_texts[int(len(padded_image_texts) * val_split) : ]

        return {"train":{"images":train_image_paths, "text": train_image_texts}, "val":{"images":val_image_paths, "text": val_image_texts}}



In [28]:
data_obj = PriMuS_Data(corpus_dirpath, corpus_filepath, dictionary_path)
lst = data_obj.trim_ds()

In [29]:
# Resize images to desired dimensions with unknown width
def resize_no_width(image, height):
    width = int(float(height * image.shape[1]) / image.shape[0])
    sample_img = cv2.resize(image, (width, height))
    return sample_img


# Pre-process a single image sample(resize/normalize/max_width resize)
def process_single_sample(img_path, label,max_width = 30):
    params = default_model_params(img_height,primus.vocabulary_size)
    # 1. Read image
    img = tf.io.read_file(img_path)
    # 2. Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=1)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize to the desired size
    img = tf.image.resize(img,[params['img_height'], max_width])
    # TODO: Get height from params, calculate width from max of all images
    return {"image": img, "label": label}


In [30]:
batch_size = 16


image_widths = [cv2.imread(img).shape[1] for img in lst["train"]["images"]]
max_image_width = max(image_widths)
print(max_image_width)
# Parameterization
img_height = 64
params = default_model_params(img_height,primus.vocabulary_size)
max_epochs = 100
dropout = 0.5
# Model
model = buildModel(params,input_width = max_image_width)
model.summary()

# Pre-process a single image sample(resize/normalize/max_width resize)
def process_single_sample(img_path, label):
    params = default_model_params(img_height,primus.vocabulary_size)
    # 1. Read image
    img = tf.io.read_file(img_path)
    # 2. Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=1)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize to the desired size
    img = tf.image.resize(img,[params['img_height'], max_image_width])
    # TODO: Get height from params, calculate width from max of all images
    return {"image": img, "label": label}
# train_dataset = train_dataset.map(process_single_sample(max_width = max_image_width))

# processed_images = [process_single_sample(img,label,max_image_width) for img,label in zip(lst["train"]["images"], lst["train"]["text"])]

# train_dataset = tf.data.Dataset.from_tensor_slices((lst['train']['images'], lst['train']['text']))

# train_dataset = (
#     train_dataset.map(
#         process_single_sample, num_parallel_calls=tf.data.experimental.AUTOTUNE
#     )
#     .batch(batch_size)
#     .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
# )

# validation_dataset = tf.data.Dataset.from_tensor_slices((lst['val']['images'], lst['val']['text']))
# validation_dataset = (
#     validation_dataset.map(
#         process_single_sample, num_parallel_calls=tf.data.experimental.AUTOTUNE
#     )
#     .batch(batch_size)
#     .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
# )
def tfdata_generator_train(batch_size= 16):
  '''Construct a data generator using `tf.Dataset`. '''
  train_dataset = tf.data.Dataset.from_tensor_slices((lst['train']['images'], lst['train']['text']))
  train_dataset = train_dataset.map(process_single_sample)
  train_dataset = train_dataset.batch(batch_size)
  # train_dataset = train_dataset.repeat()
  train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

  return train_dataset

def tfdata_generator_val(batch_size= 16):
  '''Construct a data generator using `tf.Dataset`. '''
  val_dataset = tf.data.Dataset.from_tensor_slices((lst['val']['images'], lst['val']['text']))
  val_dataset = val_dataset.map(process_single_sample)
  val_dataset = val_dataset.batch(batch_size)
  # val_dataset = val_dataset.repeat()
  val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)

  return val_dataset

2003
(None, 8, None, 64)
Model: "OMR"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 64, None, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv_1 (Conv2D)                (None, 32, None, 32  32768       ['image[0][0]']                  
                                )                                                                 
                                                                                                  
 conv_1_bn (BatchNormalization)  (None, 32, None, 32  128        ['conv_1[0][0]']                 
                                )                                      

In [31]:
train_dataset = tfdata_generator_train()
validation_dataset = tfdata_generator_val()


In [32]:
type(train_dataset)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [33]:
# Define the number of epochs.
epochs = 30

# Train the model
history = model.fit(
    train_dataset,
    # tf.compat.v1.data.make_one_shot_iterator(train_dataset).get_next(),
    # validation_data = tf.compat.v1.data.make_one_shot_iterator(validation_dataset).get_next(),
    validation_data = validation_dataset,
    epochs=epochs
)

Epoch 1/30


ValueError: in user code:

    File "c:\Users\steli\.conda\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\steli\.conda\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\steli\.conda\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\steli\.conda\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 861, in train_step
        self._validate_target_and_loss(y, loss)
    File "c:\Users\steli\.conda\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 818, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=<function CTCLoss at 0x000002D2D19084C0>, and therefore expects tar